## Notebook to simplify EEZ geometries

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas
import os
import json
# from pandarallel import pandarallel
# import topojson as tp
from zipfile import ZipFile

In [ ]:
merge = gpd.read_file('/Users/sofia/Documents/HE_Data/NRC/NRC_Marine/EEZ_boundaries/EEZ_boundaries.shp')
len(merge)

In [ ]:
# Fix topological errors
!mapshaper-xl -i snap /Users/sofia/Documents/HE_Data/NRC/NRC_Marine/EEZ_boundaries/EEZ_boundaries.shp -clean gap-fill-area=30km2 -o format=shapefile /Users/sofia/Documents/HE_Data/NRC/NRC_Marine/EEZ_corrected/EEZ_corrected.shp force

In [ ]:
m_clean = gpd.read_file('/Users/sofia/Documents/HE_Data/NRC/NRC_Marine/EEZ_corrected/EEZ_corrected.shp')
len(m_clean)

In [ ]:
sum(eez.is_valid) #186 out of 201 are valid (15 geometries invalid)

In [ ]:
eez.head()

In [ ]:
# Break the shp into 10 json files
!mapshaper-xl 16gb -i ./../../HE_Data/NRC/NRC_Marine/eez_countries_original_dissolved/*.shp no-topology combine-files \
    -merge-layers \
    -each "group = Math.trunc(this.id/26)+1" \
    -split group \
    -o EEZ_test/ format=geojson precision=0.0001 force ndjson

In [ ]:
## First test with all parameters
!mapshaper-xl 16gb -i ./EEZ_test/*.json snap \
    -simplify 10% weighting=0.3 planar keep-shapes \
    -filter-islands min-vertices=3 min-area=10000m2 remove-empty \
    -filter-slivers min-area=10000m2 remove-empty \
    -clean rewind \
    -o EEZ_test/ format=geojson force ndjson

In [ ]:
### Final test: Only simplify (retains all geometries).
!mapshaper-xl 16gb -i ./EEZ_test/*.json snap \
    -simplify 10% weighting=0.3 planar keep-shapes \
    -o EEZ_test/ format=geojson force ndjson

In [ ]:
# Combine all json files in a shp. 
!mapshaper-xl 16gb -i ./EEZ_test/*.json combine-files -merge-layers -info -o EEZ_test/simp-eez.shp

In [ ]:
sim = gpd.read_file('EEZ_test/simp-eez.shp') # len 201 ### retains all geometries
len(sim)

In [ ]:
%%time
sum(sim.is_valid)

In [ ]:
%%time
sim['geometry'] = sim['geometry'].buffer(0) # this is to repair geometries
sum(sim.is_valid)

In [ ]:
# check if new geometries in corrected database are the same as the simplified
n= 'ESP'
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
eez[eez['GID_0']==n].geometry.plot(ax = ax1) 
sim[sim['GID_0']==n].geometry.plot(ax = ax2)

In [ ]:
# check if new geometries in corrected database are the same as the simplified
n= 'GRL'
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
eez[eez['GID_0']==n].geometry.plot(ax = ax1) 
sim[sim['GID_0']==n].geometry.plot(ax = ax2)

In [ ]:
# check if new geometries in corrected database are the same as the simplified
n= 'SXM'
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
eez[eez['GID_0']==n].geometry.plot(ax = ax1) 
sim[sim['GID_0']==n].geometry.plot(ax = ax2)

In [ ]:
# check if new geometries in corrected database are the same as the simplified
n= 'SGP'
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
eez[eez['GID_0']==n].geometry.plot(ax = ax1) 
sim[sim['GID_0']==n].geometry.plot(ax = ax2)

In [ ]:
eez.crs

In [ ]:
# Save file in local
sim.to_file('/Users/sofia/Documents/HE_Data/NRC/NRC_Marine/EEZ_simplified/EEZ_simplified.shp', driver= 'ESRI Shapefile')